In [1]:
import xarray as xr
import numpy as np
from xhistogram.xarray import histogram as xhist
import matplotlib.pyplot as plt
from datetime import datetime
import sys, os, warnings
#sys.path.append(os.path.abspath("/work/bb1018/b380873/tropic_vis/utilities/"))
sys.path.append(os.path.abspath("/xdisk/sylvia/tropic_vis/utilities/"))
from plotting_utilities import *
from densityPlot import centroids
from calc_water import calc_water

In [2]:
from IPython.display import Audio, display
def allDone():
    display(Audio('/xdisk/sylvia/Hallelujah-sound-effect.mp3', autoplay=True))

In [3]:
warnings.filterwarnings( "ignore" )

### cdo commands to adjust the ICON nc variable names and contents

### Combine all ICON trajectories into 1 file

In [ ]:
# Concatenate all of the ICON-1M trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_ICON_0V1M0A0R/'
new_file = xr.open_dataset( basedir + 'traj_tst00001350_p001_trim_extract_dt_filter.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'traj_tst00001350_p' + traj_prefix(i) + str(i) + '_trim_extract_dt_filter.nc' ) # ICON
    new_file = xr.concat( (new_file, file2), dim='id' )
    
display(new_file)
new_file.to_netcdf( path=basedir + 'traj_tst00001350_trim_extract_dt_filter.nc' )

In [ ]:
# Concatenate all of the ICON-2M trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_ICON_0V2M0A0R/'
new_file = xr.open_dataset( basedir + 'traj_tst00000450_p001_trim_extract_dt_filter.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'traj_tst00000450_p' + traj_prefix(i) + str(i) + '_trim_extract_dt_filter.nc' ) # ICON
    new_file = xr.concat( (new_file, file2), dim='id' )
    
display(new_file)
new_file.to_netcdf( path=basedir + 'traj_tst00000450_trim_extract_dt_filter.nc' )

### cdo commands to adjust the CLaMS nc variables names and contents

### Combine all CLaMS trajectories into 1 file

In [ ]:
# Concatenate all of the CLAMS trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_CLAMS-Tf_0V2M0A0R/'
new_file = xr.open_dataset( basedir + 'cirrus_tst00000450_p001_trim_extract_clams_dt_iwc_filter.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'cirrus_tst00000450_p' + traj_prefix(i) + str(i) + '_trim_extract_clams_dt_iwc_filter.nc' )
    new_file = xr.concat( (new_file, file2), dim='NPARTS' )
    
#display(new_file)
new_file.to_netcdf( path=basedir + 'cirrus_tst00000450_trim_extract_clams_dt_iwc_filter.nc' )

In [ ]:
# Concatenate all of the CLAMS trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_CLAMS-Tf_0V1M0A0R/'
new_file = xr.open_dataset( basedir + 'cirrus_tst00001350_p001_trim_extract_clams_dt_iwc_filter.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'cirrus_tst00001350_p' + traj_prefix(i) + str(i) + '_trim_extract_clams_dt_iwc_filter.nc' )
    new_file = xr.concat( (new_file, file2), dim='NPARTS' )
    
#display(new_file)
new_file.to_netcdf( path=basedir + 'cirrus_tst00001350_trim_extract_clams_dt_iwc_filter.nc' )

### Load all trajectories for 5 simulations from 5 files
Initiate the binning for different variables here as well.

In [4]:
# These thresholds are used throughout. Values below are considered negligible.
qi_threshold = 10**(-8)
Ni_threshold = 10**(-8)
RHi_threshold = 0

In [5]:
basedir = '/xdisk/sylvia/traj_output/'
fichiers = [ 'CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc', 'ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc',
             'CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc', 'ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter.nc',
             'CLAMS_0V2M0A0R_tst00000450_trim_extract_dt_iwc.nc', 'CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc' ]

# Logarithmic IWC bins in ppmv, as well as their centerpoints
qi_bins = np.logspace( -5, 3, 50 )
qi_bins_c = ( qi_bins[1:] + qi_bins[:-1] )/2.

# Logarithmic Ni bins in L-1, as well as their centerpoints
Ni_bins = np.logspace( -4, 4, 50 )
Ni_bins_c = ( Ni_bins[1:] + Ni_bins[:-1] )/2.

# Linear T bins in K
T_bins = np.linspace( 190, 240, 50 )
T_bins_c = ( T_bins[1:] + T_bins[:-1] )/2.

# Linear RHi bins in %
RHi_bins = np.linspace( 60, 120, 50 )
RHi_bins_c = ( RHi_bins[1:] + RHi_bins[:-1] )/2.

# Flight 7 track times
time0 = datetime(2017, 8, 8, 6, 20)
timef = datetime(2017, 8, 8, 6, 48)

### Create a generically structured dataset to store histogram values and copy it 5 times

In [6]:
# Initially all the datasets contain the same variables so we can create a generic dataset...
ds_generic = xr.Dataset( 
    data_vars=dict(  qih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     Th=(['T_bin'], np.zeros(T_bins_c.shape)),
                     qih_outflow=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_outflow=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qih_insitu=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_insitu=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qih_flight=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_flight=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     RHih=(['RHi_bin'], np.zeros(RHi_bins_c.shape)) ),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c),
                  T_bin=(['T_bin'], T_bins_c),
                  RHi_bin=(['RHi_bin'], RHi_bins_c) ) )

# ... and then copy it 6 times.
datasets =  [ ds_generic, ds_generic.copy(), ds_generic.copy(), ds_generic.copy(), ds_generic.copy(), ds_generic.copy() ]
del ds_generic

## Different 'flavors' of qi and Ni histograms calculated and mean / median / std qi / Ni printed

In [7]:
RHi_1M0T1S = xr.open_dataset( basedir + fichiers[0] )['RHI']
RHi_2M1T1S = xr.open_dataset( basedir + fichiers[2] )['RHI']
RHi_2M0T1S = xr.open_dataset( basedir + fichiers[4] )['RHI']
RHi_2M1T0S = xr.open_dataset( basedir + fichiers[5] )['RHI']
RHi_fi = [ RHi_1M0T1S, RHi_1M0T1S, RHi_2M1T1S, RHi_2M1T1S, RHi_2M0T1S, RHi_2M1T0S ]

In [ ]:
%%time
# Save the qih_* values in the Datasets above
means = np.zeros( (6,3) )
medians = np.zeros( (6,3) )
stds = np.zeros( (6,3) )

for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    #k = 700 #2000
    fi = xr.open_dataset( basedir + f )
    
    # Take only the first 2000 trajectories to see if bimodality is caused by different trajs k=2000 and [:k]
    # Take only the first 700 timepoints to see if bimodality is caused by different times k=700 and [:,:k]
    qi = fi['qi']
    T = fi['T']
    if (j != 1) & (j != 3):
        P = fi['PE']
    else:
        P = fi['p']
    time = fi['time']
    
    # Convert kg kg-1 to ppmv
    qi_ppmv = calc_water( T, P, qi )
    qi_ppmv.name = 'qi_ppmv'
    # Ensure that qi [kg kg-1] is not mistakenly used below
    del qi

    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    qi_ppmv = qi_ppmv.where( (r > RHi_threshold ) & (qi_ppmv > qi_threshold) )
    qi_outflow = qi_ppmv.where( (T >= 210) & (T <= 237) )
    qi_insitu = qi_ppmv.where( (T < 210) )
    #qi_flight = qi_ppmv.sel( time=slice(time0, timef) )
    
    # Set the qi statistics
    #means[j,0] = np.nanmean(qi_ppmv)
    #medians[j,0] = np.nanmedian(qi_ppmv)
    #stds[j,0] = np.nanstd(qi_ppmv)
    #means[j,1] = np.nanmean(qi_outflow)
    #medians[j,1] = np.nanmedian(qi_outflow)
    #stds[j,1] = np.nanstd(qi_outflow)
    #means[j,2] = np.nanmean(qi_insitu)
    #medians[j,2] = np.nanmedian(qi_insitu)
    #stds[j,2] = np.nanstd(qi_insitu)
    print(np.nanstd(T),np.nanstd(T_outflow),np.nanstd(T_insitu),np.nanstd(T_flight))
    #print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_outflow, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_insitu, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih_insitu'] = h
    
    #wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    #wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    #h = xhist( qi_flight, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    #d['qih_flight'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, T )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( T, dim=['time','id'], block_size=100, weights=wgts, bins=[T_bins] )
    d['Th'] = h

In [ ]:
%%time
# Save the Nih_* values in the Datasets above.
means = np.zeros( (6,3) )
medians = np.zeros( (6,3) )
stds = np.zeros( (6,3) )

for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    #k = 700
    if j == 1:
        print('ICON 1-mom.. geht es weiter!')
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        continue
    else:
        fi = xr.open_dataset( basedir + f )
    
        # Convert kg-1 to L-1
        rho = fi['rho'] #[:k]
        Ni = fi['Ni'] * rho  / 1000.
        Ni.name = fi['Ni'].name
        T = fi['T']
        time = fi['time']
    
        # Filter for non-negligible values
        Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
        Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
        Ni_insitu = Ni.where( (T < 210) )
        Ni_flight = Ni.sel( time=slice(time0, timef) )
        
        # Set the qi statistics
        #means[j,0] = np.nanmean(Ni)
        #medians[j,0] = np.nanmedian(Ni)
        #stds[j,0] = np.nanstd(Ni)
        #means[j,1] = np.nanmean(Ni_outflow)
        #medians[j,1] = np.nanmedian(Ni_outflow)
        #stds[j,1] = np.nanstd(Ni_outflow)
        #means[j,2] = np.nanmean(Ni_insitu)
        #medians[j,2] = np.nanmedian(Ni_insitu)
        #stds[j,2] = np.nanstd(Ni_insitu)
        #print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_outflow, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_insitu, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_flight, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_flight'] = h

### TE and RHI histograms calculated

In [ ]:
%%time
basedir = '/xdisk/sylvia/traj_output/'

# Save the RHIh_ values for both datasets and the TEh_* values for the CLaMS Datasets only
for f, d, j in zip(fichiers, datasets, np.arange(6)):
    print(f,j)
    #k = 700
    fi = xr.open_dataset( basedir + f )
    qi = fi['qi']  #[:k]
    
    # Convert kg kg-1 to ppmv
    T = fi['T']
    if (j != 1) and (j != 3):
        P = fi['PE']
    else:
        P = fi['p']
    qi_ppmv = calc_water( T, P, qi )
    qi_ppmv.name = 'qi_ppmv'
    # Ensure that qi [kg kg-1] is not mistakenly used below
    del qi
    
    RHi = fi['RHI'].where( (qi_ppmv > qi_threshold) )
    wgts = 1*xr.apply_ufunc( np.isfinite, RHi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( RHi, dim=['time','id'], block_size=100, weights=wgts, bins=[RHi_bins] )
    d['RHih'] = h
    
for f, d, j in zip(fichiers, datasets, np.arange(6)):
    print(f,j)
    #k = 700
    fi = xr.open_dataset( basedir + f )
    qi = fi['qi']
    
    # Convert kg kg-1 to ppmv
    T = fi['T']
    if (j != 1) and (j != 3):
        P = fi['PE']
    else:
        P = fi['p']
    qi_ppmv = calc_water( T, P, qi )
    qi_ppmv.name = 'qi_ppmv'
    # Ensure that qi [kg kg-1] is not mistakenly used below
    del qi
    
    if (j != 1) and (j != 3):
        T = fi['TE'].where( qi_ppmv > qi_threshold )
    else:
        T = fi['T'].where( qi_ppmv > qi_threshold )
        
    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, T )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( T, dim=['time','id'], block_size=100, weights=wgts, bins=[T_bins] )
    d['TEh'] = h

### qi-Nih-T-RHi 1D histograms saved to nc files

In [23]:
for d, j in zip(datasets, np.arange(6)):
    d['qih'].attrs['description'] = "1D hist values of ice mass mixing ratio across qi_bin" 
    d['qih'].attrs['units'] = "Probability [%]"
    d['Nih'].attrs['description'] = "1D hist values of ice crystal number concentration across Ni_bin" 
    d['Nih'].attrs['units'] = "Probability [%]"
    d['Th'].attrs['description'] = "1D hist values of temperature across T_bin" 
    d['Th'].attrs['units'] = "Probability [%]"
    d['qih_outflow'].attrs['description'] = "1D hist values of ice mass mixing ratio for T >= 210 K across qi_bin" 
    d['qih_outflow'].attrs['units'] = "Probability [%]"
    d['Nih_outflow'].attrs['description'] = "1D hist values of ice crystal number concentration for T >= 210 K across Ni_bin" 
    d['Nih_outflow'].attrs['units'] = "Probability [%]"
    d['qih_insitu'].attrs['description'] = "1D hist values of ice mass mixing ratio for T < 210 K across qi_bin" 
    d['qih_insitu'].attrs['units'] = "Probability [%]"
    d['Nih_insitu'].attrs['description'] = "1D hist values of ice crystal number concentration for T < 210 K across Ni_bin" 
    d['Nih_insitu'].attrs['units'] = "Probability [%]"
    d['qih_flight'].attrs['description'] = "1D hist values of ice mass mixing ratio for StratoClim Flight 7 times across qi_bin" 
    d['qih_flight'].attrs['units'] = "Probability [%]"
    d['Nih_flight'].attrs['description'] = "1D hist values of ice crystal number concentration for StratoClim Flight 7 times across Ni_bin" 
    d['Nih_flight'].attrs['units'] = "Probability [%]"
    d['RHih'].attrs['description'] = "1D hist values of relative humidity wrt ice across RHI_bin" 
    d['RHih'].attrs['units'] = "Probability [%]"
    if j == 0 or j == 2 or j == 4 or j == 5:
        d['TEh'].attrs['description'] = "1D hist values of external temperature (includes latent heating) across TE_bin (only for CLaMS)"
        d['TEh'].attrs['units']="Probability [%]"

# Ice microphysical process histograms
These differ between CLaMS and ICON so we have to initially create two different structures.

In [6]:
# For CLaMS, we will look at the homogeneous, heterogeneous, and preexisiting qi and Ni
ds_clams = xr.Dataset( 
    data_vars=dict(  qhomih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nhomih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qhetih=(['qi_bin'], np.zeros(T_bins_c.shape)),
                     Nhetih=(['Ni_bin'], np.zeros(qi_bins_c.shape)),
                     qpreih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Npreih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)) ),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c) ) )

# ... and then copy it 4 times.
datasets_clams = [ ds_clams, ds_clams.copy() ]#, ds_clams.copy(), ds_clams.copy() ]
del ds_clams

In [10]:
clams_fichiers = [ fichiers[0], fichiers[2] ]#, fichiers[4], fichiers[5] ]
for f, d in zip(clams_fichiers, datasets_clams):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert to ppmv / L-1 and filter for non-negligible values
    qi_hom = fi['IWC_hom']
    P = fi['PE']
    T = fi['T']
    # Convert kg kg-1 to ppmv
    qi_hom_ppmv = calc_water( T, P, qi_hom )
    qi_hom_ppmv.name = 'qi_hom_ppmv'
    # Ensure that qi_hom [kg kg-1] is not mistakenly used below
    del qi_hom
    qi_hom_ppmv = qi_hom_ppmv.where( qi_hom_ppmv > qi_threshold )
    
    qi_het = fi['IWC_het']
    # Convert kg kg-1 to ppmv
    qi_het_ppmv = calc_water( T, P, qi_het )
    qi_het_ppmv.name = 'qi_het_ppmv'
    # Ensure that qi_het [kg kg-1] is not mistakenly used below
    del qi_het
    qi_het_ppmv = qi_het_ppmv.where( qi_het_ppmv > qi_threshold )
    
    qi_pre = fi['IWC_pre']
    # Convert kg kg-1 to ppmv
    qi_pre_ppmv = calc_water( T, P, qi_pre )
    qi_pre_ppmv.name = 'qi_pre_ppmv'
    # Ensure that qi_pre [kg kg-1] is not mistakenly used below
    del qi_pre
    qi_pre_ppmv = qi_pre_ppmv.where( qi_pre_ppmv > qi_threshold )
    
    rho = fi['rho']
    Ni_hom = fi['ICN_hom'] * rho / 1000
    Ni_hom.name = fi['ICN_hom'].name
    Ni_hom = Ni_hom.where( Ni_hom > Ni_threshold )
    
    Ni_het = fi['ICN_het'] * rho / 1000
    Ni_het.name = fi['ICN_het'].name
    Ni_het = Ni_hom.where( Ni_het > Ni_threshold )
    
    Ni_pre = fi['ICN_pre'] * rho / 1000
    Ni_pre.name = fi['ICN_pre'].name
    Ni_pre = Ni_pre.where( Ni_pre > Ni_threshold )

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_hom_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_hom_ppmv, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qhomih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_het_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_het_ppmv, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qhetih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_pre_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_pre_ppmv, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qpreih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_hom )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_hom, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nhomih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_het )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_het, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nhetih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_pre )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_pre, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Npreih'] = h

CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc
CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc


In [11]:
for d in datasets_clams:
    d['qhomih'].attrs['description'] = "1D hist values of ice mass mixing ratio from homogeneous nucleation across qi_bin" 
    d['qhomih'].attrs['units'] = "Probability [%]"
    d['Nhomih'].attrs['description'] = "1D hist values of ice crystal number concentration from homogeneous nucleation across Ni_bin" 
    d['Nhomih'].attrs['units'] = "Probability [%]"
    d['qhetih'].attrs['description'] = "1D hist values of ice mass mixing ratio from heterogeneous nucleation across qi_bin" 
    d['qhetih'].attrs['units'] = "Probability [%]"
    d['Nhetih'].attrs['description'] = "1D hist values of ice crystal number concentration from heterogeneous nucleation across Ni_bin" 
    d['Nhetih'].attrs['units'] = "Probability [%]"
    d['qpreih'].attrs['description'] = "1D hist values of ice mass mixing ratio from preexisting ice across qi_bin" 
    d['qpreih'].attrs['units'] = "Probability [%]"
    d['Npreih'].attrs['description'] = "1D hist values of ice crystal number concentration from preexisting ice across Ni_bin" 
    d['Npreih'].attrs['units'] = "Probability [%]"
    
#for d in datasets_icon:
#    d['qsedih'].attrs['description'] = "1D hist values of ice mass mixing ratio sedimenting in across qi_bin" 
#    d['qsedih'].attrs['units'] = "Probability [%]"
#    d['Nsedih'].attrs['description'] = "1D hist values of ice crystal number concentration sedimenting in across Ni_bin" 
#    d['Nsedih'].attrs['units'] = "Probability [%]"
#    d['qsedoh'].attrs['description'] = "1D hist values of ice mass mixing ratio sedimenting out across qi_bin" 
#    d['qsedoh'].attrs['units'] = "Probability [%]"
#    d['Nsedoh'].attrs['description'] = "1D hist values of ice crystal number concentration sedimenting out across Ni_bin" 
#    d['Nsedoh'].attrs['units'] = "Probability [%]"

In [14]:
writedir = '/work/bb1018/b380873/traj_output/traj_pp/'
writedir = '/xdisk/sylvia/traj_output/traj_pp/'
names = [ 'het-hom-pre-CLAMS-Tf_0V1M0A0R.nc', 'het-hom-pre-CLAMS-Tf_0V2M0A0R.nc' ]#, 'sed-ICON_0V2M0A0R.nc',
         #'het-hom-pre-CLAMS_0V2M0A0R.nc', 'het-hom-pre-CLAMS-Tf_0V2M0A0R_noSHflux.nc' ]
datasets = [ datasets_clams[0], datasets_clams[1] ]#, datasets_icon[0], datasets_clams[2], datasets_clams[3] ]

for n, d in zip(names, datasets):
    d.to_netcdf( writedir + n )

## Two-dimensional histograms - qi, Ni versus T, RHi

In [8]:
# Reduce the number of qi and Ni bins relative to the 1D histograms

# Logarithmic IWC bins in ppmv, as well as their centerpoints
qi_bins = np.logspace( -5, 3.5, 25 )
qi_bins_c = ( qi_bins[1:] + qi_bins[:-1] )/2.

# Logarithmic Ni bins in cm-3, as well as their centerpoints
Ni_bins = np.logspace( -4, 5.5, 25 )
Ni_bins_c = ( Ni_bins[1:] + Ni_bins[:-1] )/2.

### Using StratoClim data

In [ ]:
sc = xr.open_dataset( '/xdisk/sylvia/tropic_vis/obs/stratoclim2017.geophysika.0808_1.filtered_per_sec.nc')
sc_qi = sc['BEST:IWC']
sc_temp = sc['BEST:TEMP']

sc_qi = sc_qi.where( (sc_qi > qi_threshold) )
sc_temp = sc_temp.where( (sc_qi > qi_threshold) )

wgts = 1*xr.apply_ufunc( np.isfinite, sc_qi )
wgts = wgts / wgts.sum( dim=['time'] ) * 100.

h = xhist( sc_qi, sc_temp, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
qs = qi_bins_c.shape[0]
ts = T_bins_c.shape[0]
ds_generic = xr.Dataset( data_vars=dict( qiTh_flight=(['qi_bin','T_bin'], np.zeros((qs, ts)) ) ) )
ds_generic['qiTh_flight'] = h
ds_generic.to_netcdf( '/xdisk/sylvia/tropic_vis/obs/stratoclim2017.geophysika.0808_1_hist.nc' )

### Using simulation trajectories

In [9]:
# Initially all the datasets contain the same variables so we can create a generic dataset...
qs = qi_bins_c.shape[0]
ts = T_bins_c.shape[0]
ns = Ni_bins_c.shape[0]
rs = RHi_bins_c.shape[0]

ds_generic = xr.Dataset( 
    data_vars=dict(  qiTh=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_outflow=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_outflow=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_outflow=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_outflow=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_insitu=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_insitu=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_insitu=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_insitu=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_flight=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_flight=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_flight=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_flight=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c),
                  T_bin=(['T_bin'], T_bins_c),
                  RHi_bin=(['RHi_bin'], RHi_bins_c)) )

# ... and then copy it 6 times.
datasets = [ ds_generic , ds_generic.copy(), ds_generic.copy(), ds_generic.copy() ]#, ds_generic.copy(), ds_generic.copy() ]
del ds_generic

In [9]:
for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg kg-1 to ppmv
    qi = fi['qi']
    T = fi['T']
    if (j != 1) & (j != 3):
        P = fi['PE']
    else:
        P = fi['p']
    time = fi['time']
    qi_ppmv = calc_water( T, P, qi )
    qi_ppmv.name = 'qi_ppmv'
    # Ensure that qi [kg kg-1] is not mistakenly used below
    del qi
   
    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    qi_ppmv = qi_ppmv.where( (r > RHi_threshold ) & (qi_ppmv > qi_threshold) )
    T = T.where( (r > RHi_threshold ) & (qi_ppmv > qi_threshold) )
    #print('ALL')
    #print(np.nanmean(qi_ppmv), np.nanmean(T))
    
    qi_ppmv_outflow = qi_ppmv.where( (T >= 210) & (T <= 237) )
    T_outflow = T.where( (T >= 210) & (T <= 237) )
    #print('OUTFLOW')
    #print(np.nanmean(qi_ppmv_outflow), np.nanmean(T_outflow))
    
    qi_ppmv_insitu = qi_ppmv.where( (T < 210) )
    T_insitu = T.where( (T < 210) )
   
    qi_flight = qi_ppmv.sel( time=slice(time0, timef) )
    T_flight = T.sel( time=slice(time0, timef) )

    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv, T, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv_outflow, T_outflow, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv_insitu, T_insitu, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_flight, T_flight, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_flight'] = h
    
allDone()

CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc
ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc
CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc
ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter.nc
CLAMS_0V2M0A0R_tst00000450_trim_extract_dt_iwc.nc
CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc


In [10]:
%%time
# Save the Nih_* values in the Datasets above.
for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    if j == 1:
        print('ICON 1-mom.. geht es weiter!')
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        continue
    else:
        # Convert kg-1 to L-1
        rho = fi['rho']
        Ni = fi['Ni'] * rho  / 1000.
        Ni.name = fi['Ni'].name
        T = fi['T']
        RHI = fi['RHI']
        time = fi['time']
    
        # Filter for non-negligible values
        Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
        RHI = RHI.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
        #Ni = Ni.where( (Ni > Ni_threshold) )
        #RHI = RHI.where( (Ni > Ni_threshold) )
        #print('ALL')
        #print(np.nanmean(Ni), np.nanmedian(Ni))
        #print(np.nanmean(RHI), np.nanmedian(RHI))
    
        Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
        RHI_outflow = RHI.where( (T >= 210) & (T <= 237) )
        #print('OUTFLOW')
        #print(np.nanmean(Ni_outflow), np.nanmedian(Ni_outflow))
        #print(np.nanmean(RHI_outflow), np.nanmedian(RHI_outflow))
    
        Ni_insitu = Ni.where( (T < 210) )
        RHI_insitu = RHI.where( (T < 210) )
        #print('IN-SITU')
        #print(np.nanmean(Ni_insitu), np.nanmedian(Ni_insitu))
        #print(np.nanmean(RHI_insitu), np.nanmedian(RHI_insitu))
    
        Ni_flight = Ni.sel( time=slice(time0, timef) )
        RHI_flight = RHI.sel( time=slice(time0, timef) )
        #print('FLIGHT')
        #print(np.nanmean(Ni_flight), np.nanmedian(Ni_flight))
        #print(np.nanmean(RHI_flight), np.nanmedian(RHI_flight))
        #print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
        # Weight by the total number of non-nan values
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni, RHI, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
        d['NiRHih'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_outflow, RHI_outflow, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
        d['NiRHih_outflow'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_insitu, RHI_insitu, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
        d['NiRHih_insitu'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_flight, RHI_flight, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
        d['NiRHih_flight'] = h
    
allDone()

CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc
cov 23.568650849029975
pearson 0.0002336006602890353
Ellipse major axis: <xarray.DataArray 'RHI' ()>
array(30.74302978) K
Min x value of major axis: <xarray.DataArray 'RHI' ()>
array(83.41756352) K
Max x value of major axis: <xarray.DataArray 'RHI' ()>
array(114.16059329) K
Affine2D(
    [[ 1.08693024e+01 -1.08693024e+01  9.87890784e+01]
     [ 4.64118616e+03  4.64118616e+03  5.02552477e+02]
     [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]])
ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc
ICON 1-mom.. geht es weiter!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc
cov -51.81755650562666
pearson -0.0008488778390346616
Ellipse major axis: <xarray.DataArray 'RHI' ()>
array(27.90673033) K
Min x value of major axis: <xarray.DataArray 'RHI' ()>
array(86.22503465) K
Max x value of major axis: <xarray.DataArray 'RHI' ()>
array(114.13176498) K
Affine2D(
    [[ 9.866

KeyboardInterrupt: 

In [12]:
for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg kg-1 to ppmv
    qi = fi['qi']
    T = fi['T']
    RHI = fi['RHI']
    if (j != 1) & (j != 3):
        P = fi['PE']
    else:
        P = fi['p']
    time = fi['time']
    qi_ppmv = calc_water( T, P, qi )
    qi_ppmv.name = 'qi_ppmv'
    # Ensure that qi [kg kg-1] is not mistakenly used below
    del qi

    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    qi_ppmv = qi_ppmv.where( (qi_ppmv > qi_threshold) & (r > RHi_threshold) )
    RHI = RHI.where( (qi_ppmv > qi_threshold) & (r > RHi_threshold) )
    #print('ALL')
    #print(np.nanmean(qi_ppmv), np.nanmedian(qi_ppmv))
    #print(np.nanmean(RHI), np.nanmedian(RHI))
    
    qi_ppmv_outflow = qi_ppmv.where( (T >= 210) & (T <= 237) )
    RHI_outflow = RHI.where( (T >= 210) & (T <= 237) )
    #confidence_ellipse( x=RHI_outflow, y=qi_ppmv_outflow, n_std=1 )
    print('OUTFLOW')
    print(np.nanmean(qi_ppmv_outflow), np.nanmedian(qi_ppmv_outflow),np.nanstd(qi_ppmv_outflow))
    print(np.nanmean(RHI_outflow), np.nanmedian(RHI_outflow),np.nanstd(RHI_outflow))
    
    qi_ppmv_insitu = qi_ppmv.where( (T < 210) )
    RHI_insitu = RHI.where( (T < 210) )
    #print('INSITU')
    #print(np.nanmean(qi_ppmv_insitu), np.nanmedian(qi_ppmv_insitu))
    #print(np.nanmean(RHI_insitu), np.nanmedian(RHI_insitu))
    
    qi_ppmv_flight = qi_ppmv.sel( time=slice(time0, timef) )
    RHI_flight = RHI.sel( time=slice(time0, timef) )
    #print('FLIGHT')
    #print(np.nanmean(qi_ppmv_flight), np.nanmedian(qi_ppmv_flight))
    #print(np.nanmean(RHI_flight), np.nanmedian(RHI_flight))
    #print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`')

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv, RHI, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv_outflow, RHI_outflow, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_ppmv_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_ppmv_insitu, RHI_insitu, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih_insitu'] = h
    
    #wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    #wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    #h = xhist( qi_ppmv_flight, RHI_flight, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    #d['qiRHih_flight'] = h

allDone()

CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc
OUTFLOW
25.568006876296693 3.269421283663666 63.34760991150228
98.75754321251502 98.08903778804782 15.330531266859142
ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc
OUTFLOW
15.8944 3.4288466 26.98005
92.63503 96.488914 9.483006
CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc
OUTFLOW
60.680852238291315 11.767587862906378 125.47325601422234
100.15082649436627 99.40907335954074 13.913809841078113
ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter.nc
OUTFLOW
26.54974 1.771181 84.437874
88.59463 92.05847 13.981982


In [13]:
%%time
# Save the Nih_* values in the Datasets above.
for f, d, r, j in zip(fichiers, datasets, RHi_fi, np.arange(6)):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    if j == 1:
        print('ICON 1-mom.. geht es weiter!')
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        continue
    else:
        # Convert kg-1 to L-1
        rho = fi['rho']
        Ni = fi['Ni'] * rho  / 1000.
        Ni.name = fi['Ni'].name
        T = fi['T']
        time = fi['time']
    
        # Filter for non-negligible values
        Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
        T = T.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    
        Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
        T_outflow = T.where( (T >= 210) & (T <= 237) )
    
        Ni_insitu = Ni.where( (T < 210) )
        T_insitu = T.where( (T < 210) )
    
        Ni_flight = Ni.sel( time=slice(time0, timef) )
        T_flight = T.sel( time=slice(time0, timef) )

        # Weight by the total number of non-nan values
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni, T, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
        d['NiTh'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_outflow, T_outflow, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
        d['NiTh_outflow'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_insitu, T_insitu, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
        d['NiTh_insitu'] = h
    
        wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
        wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
        h = xhist( Ni_flight, T_flight, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
        d['NiTh_flight'] = h
        
allDone()

CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc_filter.nc
ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc
ICON 1-mom.. geht es weiter!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc
ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter.nc


CPU times: user 1min 9s, sys: 27.8 s, total: 1min 37s
Wall time: 2min 49s


### qi-Nih-T-RHi 2D histograms saved to nc files

In [21]:
for d, j in zip(datasets, np.arange(6)):
    d['qiTh'].attrs['description'] = "2D hist values of ice mass mixing ratio versus temperature over [qi_bin, T_bin]" 
    d['qiTh'].attrs['units'] = "Probability [%]"
    d['NiTh'].attrs['description'] = "2D hist values of ice crystal number concentration versus temperature over [Ni_bin, T_bin]" 
    d['NiTh'].attrs['units'] = "Probability [%]"
    d['qiTh_outflow'].attrs['description'] = "2D hist values of ice mass mixing ratio versus temperature for T >= 210 K over [qi_bin, T_bin]" 
    d['qiTh_outflow'].attrs['units'] = "Probability [%]"
    d['NiTh_outflow'].attrs['description'] = "2D hist values of ice crystal number concentration versus temperature for T >= 210 K over [Ni_bin, T_bin]" 
    d['NiTh_outflow'].attrs['units'] = "Probability [%]"
    d['qiTh_insitu'].attrs['description'] = "2D hist values of ice mass mixing ratio versus temperature for T < 210 K over [qi_bin, T_bin]" 
    d['qiTh_insitu'].attrs['units'] = "Probability [%]"
    d['NiTh_insitu'].attrs['description'] = "2D hist values of ice crystal number concentration versus temperature for T < 210 K over [Ni_bin, T_bin]" 
    d['NiTh_insitu'].attrs['units'] = "Probability [%]"
    d['qiTh_flight'].attrs['description'] = "2D hist values of ice mass mixing ratio versus temperature for StratoClim Flight 7 times over [qi_bin, T_bin]" 
    d['qiTh_flight'].attrs['units'] = "Probability [%]"
    d['NiTh_flight'].attrs['description'] = "2D hist values of ice crystal number concentration versus temperature for StratoClim Flight 7 times over [Ni_bin, T_bin]" 
    d['NiTh_flight'].attrs['units'] = "Probability [%]"
    
    d['qiRHih'].attrs['description'] = "2D hist values of ice mass mixing ratio versus rel hum wrt ice over [qi_bin, RHi_bin]" 
    d['qiRHih'].attrs['units'] = "Probability [%]"
    d['NiRHih'].attrs['description'] = "2D hist values of ice crystal number concentration versus rel hum wrt ice over [Ni_bin, RHi_bin]" 
    d['NiRHih'].attrs['units'] = "Probability [%]"
    d['qiRHih_outflow'].attrs['description'] = "2D hist values of ice mass mixing ratio versus rel hum wrt ice for T >= 210 K over [qi_bin, RHi_bin]" 
    d['qiRHih_outflow'].attrs['units'] = "Probability [%]"
    d['NiRHih_outflow'].attrs['description'] = "2D hist values of ice crystal number concentration versus rel hum wrt ice for T >= 210 K over [Ni_bin, RHi_bin]" 
    d['NiRHih_outflow'].attrs['units'] = "Probability [%]"
    d['qiRHih_insitu'].attrs['description'] = "2D hist values of ice mass mixing ratio versus rel hum wrt ice for T < 210 K over [qi_bin, RHi_bin]" 
    d['qiRHih_insitu'].attrs['units'] = "Probability [%]"
    d['NiRHih_insitu'].attrs['description'] = "2D hist values of ice crystal number concentration versus rel hum wrt ice for T < 210 K over [Ni_bin, RHi_bin]" 
    d['NiRHih_insitu'].attrs['units'] = "Probability [%]"
    d['qiRHih_flight'].attrs['description'] = "2D hist values of ice mass mixing ratio versus rel hum wrt ice for StratoClim Flight 7 times over [qi_bin, T_bin]" 
    d['qiRHih_flight'].attrs['units'] = "Probability [%]"
    d['NiRHih_flight'].attrs['description'] = "2D hist values of ice crystal number concentration versus rel hum wrt ice for StratoClim Flight 7 times over [Ni_bin, T_bin]" 
    d['NiRHih_flight'].attrs['units'] = "Probability [%]"

In [22]:
writedir = '/xdisk/sylvia/traj_output/traj_pp/'
names = [ 'qippmvNi-TRHi-2Dh-CLAMS-Tf_0V1M0A0R.nc', 'qippmvNi-TRHi-2Dh-ICON_0V1M0A0R.nc', 
          'qippmvNi-TRHi-2Dh-CLAMS-Tf_0V2M0A0R.nc', 'qippmvNi-TRHi-2Dh-ICON_0V2M0A0R.nc',
          'qippmvNi-TRHi-2Dh-CLAMS_0V2M0A0R.nc', 'qippmvNi-TRHi-2Dh-CLAMS-Tf_0V2M0A0R_noSHflux.nc' ]
for n, d in zip(names, datasets):
    d.to_netcdf( writedir + n )